In [42]:
from os import environ

input_folder = environ.get(
    'CROSSCOMPUTE_INPUT_FOLDER', 'tests/standard/input')
output_folder = environ.get(
    'CROSSCOMPUTE_OUTPUT_FOLDER', 'tests/standard/output')
log_folder = environ.get(
    'CROSSCOMPUTE_LOG_FOLDER', 'tests/standard/log')
debug_folder = environ.get(
    'CROSSCOMPUTE_DEBUG_FOLDER', 'tests/standard/debug')

In [43]:
import json
from os.path import join

settings_path = join(input_folder, 'settings.json')
d = json.load(open(settings_path, 'rt'))
survey_codes = d['survey-codes'].strip().split()
survey_codes

['x', 'y', 'z']

In [52]:
asset_dictionary_by_id = {}
asset_feature_by_id = {}

def absorb_path(path, survey_code):
    d = json.load(open(path, 'rt'))
    # Deduplicate by asset name
    asset_by_id = d['assets']['assetById']
    asset_names = [_['name'] for _ in asset_by_id.values()]
    
    asset_ids = []
    for asset_id, asset_dictionary in asset_dictionary_by_id.items():
        if asset_dictionary['name'] in asset_names:
            asset_ids.append(asset_id)
    
    for asset_id in asset_ids:
        try:
            del asset_dictionary_by_id[asset_id]
        except KeyError:
            pass
        try:
            del asset_feature_by_id[asset_id]
        except KeyError:
            pass
    
    for asset_id, asset_dictionary in asset_by_id.items():
        try:
            del asset_dictionary['id']
        except KeyError:
            pass
        asset_dictionary['attributes'] = dict(asset_dictionary.get('attributes', {}), surveyCode=survey_code)
        asset_dictionary_by_id[asset_id] = asset_dictionary
    for asset_feature in d['assets']['assetsGeoJson']['features']:
        asset_id = asset_feature['properties']['id']
        asset_feature_by_id[asset_id] = asset_feature

In [53]:
from glob import glob
from os.path import expanduser, getctime

# TODO: Make sure later timestamps override earlier timestamps for overlapping asset ids

survey_folder = expanduser('~/Experiments/survey-infrastructure')
electricity_network_paths = []
for survey_code in survey_codes:
    electricity_network_paths.extend(glob(join(
        survey_folder, survey_code, '**', 'electricity-network.json',
    ), recursive=True))
for path in sorted(electricity_network_paths, key=lambda _: getctime(_)):
    absorb_path(path, survey_code)

In [63]:
asset_features = list(asset_feature_by_id.values())
asset_features;

In [64]:
electricity_network_dictionary = {
    'assets': {
        'assetById': asset_dictionary_by_id,
        'assetsGeoJson': {
            'type': 'FeatureCollection',
            'features': asset_features,
        },
        'assetTypeByCode': {
            'g': {
                'assetAttributes': [
                    ['surveyCode', survey_codes],
                ],
            },
            'l': {
                'assetAttributes': [
                    ['surveyCode', survey_codes],                    
                ],
            },
            'm': {
                'assetAttributes': [
                    ['surveyCode', survey_codes],                    
                ],
            },
        },
    },
}
electricity_network_dictionary

{'assets': {'assetById': {'IxhJEjQioBk3C6D01613061593101': {'utilityId': '',
    'typeCode': 'g',
    'name': '21/11 Emeka Anyaoku St, Garki, Abuja, Nigeria',
    'connections': {'0': {'busId': 'Vcs93sWYQd0N54AM0iJk5eVE2WjrTKfo1613061593102'}},
    'attributes': {'surveyCode': 'z'}},
   'oM4Cj67GCYHNW24O1613061610237': {'utilityId': '',
    'typeCode': 'l',
    'name': 'Tawari, Nigeria',
    'connections': {'0': {'busId': 'Vcs93sWYQd0N54AM0iJk5eVE2WjrTKfo1613061593102'},
     '1': {'busId': 'SY7bn3MRMBJbjnYkcij5YCPxhFHIJCC61613061612495'},
     '2': {'busId': '3lzlwSvDwcuZ5y4bUbI3wHMwjbKK3XCe1613061636392'}},
    'vertexCount': 3,
    'attributes': {'surveyCode': 'z'}},
   'S3BBp1830pvYe7hb1613077271568': {'utilityId': '',
    'typeCode': 'm',
    'name': 'Lagos',
    'connections': {'0': {'busId': 'dYwY3VWiLdFgpZl8JxGLDa1E1rx3kzTE1613077271569'}},
    'attributes': {'population': '10000000', 'surveyCode': 'z'}},
   'RIb6xnerRBnRRqhw1613077301139': {'utilityId': '',
    'typeCode': 'm'

In [56]:
json.dump(electricity_network_dictionary, open(join(output_folder, 'electricity-network.json'), 'wt'))

In [57]:
from collections import defaultdict

asset_count_by_type_code = defaultdict(int)
for asset_dictionary in asset_dictionary_by_id.values():
    asset_type_code = asset_dictionary['typeCode']
    asset_count_by_type_code[asset_type_code] += 1

asset_count_by_type_code

defaultdict(int, {'g': 2, 'l': 2, 'm': 4})

In [58]:
import pandas as pd
t = pd.DataFrame(asset_count_by_type_code.items(), columns=['type', 'count'])
t

,type,count
0,g,2
1,l,2
2,m,4


In [59]:
t = t.set_index('type').rename({
    'g': 'generator',
    't': 'transformer',
    'q': 'power quality',
    'm': 'meter',
    'l': 'line',
    'x': 'switch',
})

In [60]:
t.to_csv(join(output_folder, 'statistics.csv'))

In [61]:
asset_features = electricity_network_dictionary['assets']['assetsGeoJson']['features']
asset_features_by_type_code = defaultdict(list)
for asset_type_code in ['g', 'l', 'm']:
    for asset_feature in asset_features:
        if asset_feature['properties']['typeCode'] == asset_type_code:
            asset_features_by_type_code[asset_type_code].append(asset_feature)
# dict(asset_features_by_type_code)

In [62]:
import geojson

def save_map(target_path, features):
    geojson.dump({'type': 'FeatureCollection', 'features': features}, target_path)

In [33]:
save_map(open(join(output_folder, 'electricity-generators.geojson'), 'wt'), asset_features_by_type_code['g'])
save_map(open(join(output_folder, 'electricity-lines.geojson'), 'wt'), asset_features_by_type_code['l'])
save_map(open(join(output_folder, 'electricity-meters.geojson'), 'wt'), asset_features_by_type_code['m'])

In [39]:
rows = []
for asset_feature in asset_features_by_type_code['m']:
    asset_id = asset_feature['properties']['id']
    asset_dictionary = asset_dictionary_by_id[asset_id]
    name = asset_dictionary['name']
    asset_attributes = asset_dictionary.get('attributes', {})
    longitude, latitude = asset_feature['geometry']['coordinates']
    population = int(asset_attributes.get('population', 1))
    rows.append([name, latitude, longitude, population])

In [40]:
rows

[['Lagos', 6.447535210993969, 3.395496110018816, 100000],
 ['Ibadan', 7.375054588174643, 3.8946089211888504, 10000],
 ['Kano', 12.01218863743951, 8.55342944361855, 1]]

In [41]:
import csv

with open(join(output_folder, 'electricity-meters.csv'), 'wt') as output_file:
    csv_writer = csv.writer(output_file)
    csv_writer.writerow(['Name', 'Latitude', 'Longitude', 'Population'])
    csv_writer.writerows(rows)